In [ ]:
'''
TITLE: Econometric modeling of the number of points accumulated in RPL based on sports and other factors

GOAL: What indicators does football in the RPL depend on?

RESULT: [
        Mostly - by average number of goals scored, goals conceded and passes per defensive action (PPDA).
        Secondary - by quality of positional defence, how many moments team allows and how well it presses, how it copes under pressure. 
        meaning - xGA, PPDA, OPPDA and ODC are the most interesting indicators, the improvement of which teams should pay attention to, in the RPL, and if they are not Zenit.
]
'''

In [ ]:

import pandas as pd
import csv
import matplotlib.pyplot as plt
import seaborn as sns
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

from patsy import dmatrices
from statsmodels.stats.outliers_influence import variance_inflation_factor
import statsmodels.formula.api as sm
import statsmodels.api as sma
from scipy.stats import shapiro

In [ ]:
''' Creating DataFrames for each season ( 18/19; 19/20; 20/21 )'''

rfpl_19 = pd.read_csv(r'C:\Users\admin\OneDrive\Kursach/test19.csv',index_col = [0])
rfpl_20 = pd.read_csv(r'C:\Users\admin\OneDrive\Kursach/test20.csv',index_col = [0])
rfpl_21 = pd.read_csv(r'C:\Users\admin\OneDrive\Kursach/test21.csv',index_col = [0])

In [ ]:
rfpl_19

In [ ]:
rfpl_19['MEAN_PTS'] = round(rfpl_19['PTS']/rfpl_19['M'],2) 
rfpl_19['MEAN_xG'] = round(rfpl_19['xG']/rfpl_19['M'],2)
rfpl_19['MEAN_xGA'] = round(rfpl_19['xGA']/rfpl_19['M'],2) 
rfpl_19['MEAN_G'] = round(rfpl_19['G']/rfpl_19['M'],2)
rfpl_19['MEAN_GA'] = round(rfpl_19['GA']/rfpl_19['M'],2)
rfpl_19['MEAN_DC'] = round(rfpl_19['DC']/rfpl_19['M'],2)
rfpl_19['MEAN_ODC'] = round(rfpl_19['ODC']/rfpl_19['M'],2)

rfpl_20['MEAN_PTS'] = round(rfpl_20['PTS']/rfpl_20['M'],2) 
rfpl_20['MEAN_xG'] = round(rfpl_20['xG']/rfpl_20['M'],2) 
rfpl_20['MEAN_xGA'] = round(rfpl_20['xGA']/rfpl_20['M'],2) 
rfpl_20['MEAN_G'] = round(rfpl_20['G']/rfpl_20['M'],2) 
rfpl_20['MEAN_GA'] = round(rfpl_20['GA']/rfpl_20['M'],2) 
rfpl_20['MEAN_DC'] = round(rfpl_20['DC']/rfpl_20['M'],2)
rfpl_20['MEAN_ODC'] = round(rfpl_20['ODC']/rfpl_20['M'],2)


rfpl_21['MEAN_PTS'] = round(rfpl_21['PTS']/rfpl_21['M'],2) 
rfpl_21['MEAN_xG'] = round(rfpl_21['xG']/rfpl_21['M'],2) 
rfpl_21['MEAN_xGA'] = round(rfpl_21['xGA']/rfpl_21['M'],2) 
rfpl_21['MEAN_G'] = round(rfpl_21['G']/rfpl_21['M'],2) 
rfpl_21['MEAN_GA'] = round(rfpl_21['GA']/rfpl_21['M'],2) 
rfpl_21['MEAN_DC'] = round(rfpl_21['DC']/rfpl_21['M'],2)
rfpl_21['MEAN_ODC'] = round(rfpl_21['ODC']/rfpl_21['M'],2)

In [ ]:
# Parsing - team value for each season for econometric-analysis 

url19 = 'https://www.championat.com/football/_russiapl/tournament/2973/' #2019
tab19 = pd.read_html(url19)
value_team_19 = tab19[7]

url20 = 'https://www.championat.com/football/_russiapl/tournament/3953/' #2020
tab20 = pd.read_html(url20)
value_team_20 = tab20[7]

url21 = 'https://www.championat.com/football/_russiapl/tournament/4465/' #2021
tab21 = pd.read_html(url21)
value_team_21 = tab21[7]

# Creating dictionary, so df`s can be merged correctly
team_dict = {'Зенит':'Zenit St. Petersburg','ЦСКА':'CSKA Moscow','Краснодар':'FC Krasnodar',
             'Спартак М':'Spartak Moscow','Локомотив М':'Lokomotiv Moscow','Динамо М':'Dinamo Moscow',
             'Рубин':'Rubin Kazan','Сочи':'PFC Sochi','Арсенал':'Arsenal Tula',
             'Ростов':'FC Rostov','Ахмат':'FK Akhmat','Химки':'Khimki',
             'Ротор':'FC Rotor Volgograd','Урал':'Ural','Уфа':'FC Ufa',
             'Тамбов':'FC Tambov','Нижний Новгород':'Nizhny Novgorod','Крылья Советов':'Krylya Sovetov Samara',
             'Пари Нижний Новгород':'Nizhny Novgorod','Оренбург':'FC Orenburg','Торпедо М':'Torpedo Moscow','Факел':'Fakel'}

value_team_19 = value_team_19.replace({'Команда': team_dict})
value_team_20 = value_team_20.replace({'Команда': team_dict})
value_team_21 = value_team_21.replace({'Команда': team_dict})

# Changing names to EN 

value_team_19.rename(columns={"Команда": "Team", "Стоимость, €": "value_euro","Игроки":"num_players","Среднее, €":"value_per_player"},inplace = True)
value_team_20.rename(columns={"Команда": "Team", "Стоимость, €": "value_euro","Игроки":"num_players","Среднее, €":"value_per_player"},inplace = True)
value_team_21.rename(columns={"Команда": "Team", "Стоимость, €": "value_euro","Игроки":"num_players","Среднее, €":"value_per_player"},inplace = True)

# Deleting spaces between str-values
value_team_19['value_euro'] = value_team_19['value_euro'].str.replace(' ', '')
value_team_19['value_per_player'] = value_team_19['value_per_player'].str.replace(' ', '')

value_team_20['value_euro'] = value_team_20['value_euro'].str.replace(' ', '')
value_team_20['value_per_player'] = value_team_20['value_per_player'].str.replace(' ', '')

value_team_21['value_euro'] = value_team_21['value_euro'].str.replace(' ', '')
value_team_21['value_per_player'] = value_team_21['value_per_player'].str.replace(' ', '')

# Converting str-columns into float

value_team_19 = value_team_19.astype({'value_euro': 'float64','value_per_player': 'float64'})
value_team_20 = value_team_20.astype({'value_euro': 'float64','value_per_player': 'float64'})
value_team_21 = value_team_21.astype({'value_euro': 'float64','value_per_player': 'float64'})

In [ ]:
# Merging original and parsed DF`s by team name
rfpl_19 = pd.merge(rfpl_19, value_team_19, on='Team')
rfpl_20 = pd.merge(rfpl_20, value_team_20, on='Team')
rfpl_21 = pd.merge(rfpl_21, value_team_21, on='Team')

In [ ]:
rfpl_19

In [ ]:
# Merging them all into one df
df = pd.concat([rfpl_19, rfpl_20, rfpl_21],ignore_index = True)

In [ ]:
# Creating list with variables
dependent_var = ['PTS', 'MEAN_xG','MEAN_xGA', 'MEAN_G','MEAN_GA', 'value_euro','num_players','value_per_player', 'PPDA','OPPDA','MEAN_DC','MEAN_ODC']

In [ ]:
# Heatmapping them to look for false correlation and multycollinearity
sns.heatmap(data = df[dependent_var].corr(), annot = True)

In [ ]:
# Creating list with variables for two models, based on expected results and real results
xdependent_var = ['PTS', 'MEAN_xG','MEAN_xGA','value_euro', 'PPDA','OPPDA','MEAN_DC','MEAN_ODC']
rdependent_var = ['PTS', 'MEAN_G','MEAN_GA','value_euro', 'PPDA','OPPDA','MEAN_DC','MEAN_ODC']

In [ ]:
'''
xG-MODEL
'''

In [ ]:
# Heatmapping xg-model variables to look for false correlation and multycollinearity
sns.heatmap(data = df[xdependent_var].corr(), annot = True)

In [ ]:
# Creating Matrix of multilinear regression with PTS as dependent-variable and others as independent
y, X = dmatrices('PTS ~ MEAN_xG + MEAN_xGA + value_euro + PPDA + OPPDA + MEAN_DC + MEAN_ODC', data=df[xdependent_var], return_type='dataframe')

# Calculating Variance inflation factor
vif_exp = pd.DataFrame()
vif_exp['VIF'] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]
vif_exp['variable'] = X.columns
vif_exp

In [ ]:
# Removing MEAN_xG because of a strong multicollinearity it causes
y, X = dmatrices('PTS ~ MEAN_xGA + value_euro + PPDA + OPPDA + MEAN_DC + MEAN_ODC', data=df[xdependent_var], return_type='dataframe')

# Calculating Variance inflation factor
vif_exp = pd.DataFrame()
vif_exp['VIF'] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]
vif_exp['variable'] = X.columns
vif_exp

In [ ]:
# Parameters and regression staistics
result = sm.ols(formula="PTS ~ MEAN_xGA + value_euro + PPDA + OPPDA + MEAN_DC + MEAN_ODC", data=df[xdependent_var]).fit()
print(result.params)
print(result.summary())

In [ ]:
# Removing MEAN_DC because of t-statistics
result = sm.ols(formula="PTS ~ MEAN_xGA + value_euro + PPDA + OPPDA + MEAN_ODC", data=df[xdependent_var]).fit()
print(result.params)
print(result.summary())

In [ ]:
# Removing value-euro
result = sm.ols(formula="PTS ~ MEAN_xGA + PPDA + OPPDA + MEAN_ODC", data=df[xdependent_var]).fit()
print(result.params)
print(result.summary())

In [ ]:
# Testing for Homoscedacity
x_var = ['MEAN_xGA', 'PPDA','OPPDA','MEAN_ODC']
sma.stats.diagnostic.het_goldfeldquandt(y = df['PTS'], x=df[x_var], drop=0.2)

In [ ]:
'''
real G-MODEL
'''

In [ ]:
# Heatmapping g-model variables to look for false correlation and multycollinearity
sns.heatmap(data = df[rdependent_var].corr(), annot = True)

In [ ]:
# Creating Matrix of multilinear regression with PTS as dependent-variable and others as independent
y, X = dmatrices('PTS ~ MEAN_G + MEAN_GA + value_euro + PPDA + OPPDA + MEAN_DC + MEAN_ODC', data=df[rdependent_var], return_type='dataframe')

# Calculating Variance inflation factor
vif_exp = pd.DataFrame()
vif_exp['VIF'] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]
vif_exp['variable'] = X.columns
vif_exp

In [ ]:
# Removing MEAN_DC because of a strong multicollinearity it causes
y, X = dmatrices('PTS ~ MEAN_G + MEAN_GA + value_euro + PPDA + OPPDA + MEAN_ODC', data=df[rdependent_var], return_type='dataframe')

# Calculating Variance inflation factor
vif_exp = pd.DataFrame()
vif_exp['VIF'] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]
vif_exp['variable'] = X.columns
vif_exp

In [ ]:
# Parameters and regression staistics
result = sm.ols(formula="PTS ~ MEAN_G + MEAN_GA + value_euro + PPDA + OPPDA + MEAN_ODC", data=df[rdependent_var]).fit()
print(result.params)
print(result.summary())

In [ ]:
# Removing OPPDA
result = sm.ols(formula="PTS ~ MEAN_G + MEAN_GA + value_euro + PPDA + MEAN_ODC", data=df[rdependent_var]).fit()
print(result.params)
print(result.summary())

In [ ]:
# Removing value_euro
result = sm.ols(formula="PTS ~ MEAN_G + MEAN_GA + PPDA + MEAN_ODC", data=df[rdependent_var]).fit()
print(result.params)
print(result.summary())

In [ ]:
# Removing MEAN_ODC
result = sm.ols(formula="PTS ~ MEAN_G + MEAN_GA + PPDA ", data=df[rdependent_var]).fit()
print(result.params)
print(result.summary())

In [ ]:
# Testing for Homoscedacity
r_var = ['MEAN_G','MEAN_GA', 'PPDA']
sma.stats.diagnostic.het_goldfeldquandt(y = df['PTS'], x=df[r_var], drop=0.2)